# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
import nltk
nltk.download(['punkt','wordnet','stopwords'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# import libraries
import pandas as pd
import numpy as np
import re
import joblib
from sqlalchemy import create_engine 

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.utils.multiclass import type_of_target

In [5]:
# load data from database
engine = create_engine('sqlite:///Project3.db')
df = pd.read_sql_table('DisasterData',engine)
X=df.iloc[:,1]
Y=df.iloc[:,4:]


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    # remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]"," ",text)
    # tokenize text into words
    tokens = nltk.word_tokenize(text)
    # remove stop words
    tokens = [x for x in tokens if x not in stopwords.words("english")]
    lemmatizer=WordNetLemmatizer()
    clean_tokens=[]
    for tok in tokens:
        clean_tok=lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                   ('clf',MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
# split data into train and test sets
X_train, X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=42 )

# train pipeline

pipeline.fit(X_train,y_train)
    

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred=pipeline.predict(X_test)
category_names=Y.columns.values
print (classification_report(y_test,y_pred,target_names=category_names))

                        precision    recall  f1-score   support

               related       1.00      0.03      0.07        58
               request       0.80      0.41      0.54      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.73      0.61      0.67      3219
          medical_help       0.49      0.08      0.14       638
      medical_products       0.73      0.08      0.14       418
     search_and_rescue       0.75      0.05      0.09       192
              security       0.00      0.00      0.00       144
              military       0.54      0.09      0.15       245
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.27      0.41       500
                  food       0.85      0.41      0.55       878
               shelter       0.76      0.33      0.46       705
              clothing       0.80      0.10      0.18       115
                 money       0.80      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [164]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [18]:
parameters = {'vect__ngram_range':((1,1),(1,2)),
              'vect__max_df':(0.5,0.75,1.0)}
cv = GridSearchCV(estimator=pipeline,param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
# fit the model
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
# test the model
y_pred_cv = cv.predict(X_test)
category_names=Y.columns.values
print(classification_report(y_test,y_pred_cv,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.67      0.03      0.07        58
               request       0.79      0.47      0.59      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.74      0.53      0.62      3219
          medical_help       0.56      0.10      0.17       638
      medical_products       0.79      0.08      0.15       418
     search_and_rescue       0.82      0.05      0.09       192
              security       0.20      0.01      0.01       144
              military       0.45      0.04      0.07       245
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.28      0.42       500
                  food       0.86      0.51      0.64       878
               shelter       0.79      0.26      0.39       705
              clothing       0.59      0.09      0.15       115
                 money       0.89      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [26]:
from sklearn.tree import DecisionTreeClassifier

pipeline_new = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
                    ('tfidf',TfidfTransformer()),
                   ('clf',MultiOutputClassifier(DecisionTreeClassifier()))])
#find better parameters 
pipeline_new.get_params().keys()
parameter_tree={'clf__estimator__criterion':['gini'],
                'clf__estimator__max_depth':[2,4,6]}
cv_new=GridSearchCV(estimator=pipeline_new,param_grid=parameter_tree)

# train the new pipeline
cv_new.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__criterion': ['gini'], 'clf__estimator__max_depth': [2, 4, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
# test the pipeline model

category_names=Y.columns.values
y_pred_tree=cv_new.predict(X_test)
print(classification_report(y_test,y_pred_tree,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.73      0.14      0.23        58
               request       0.79      0.41      0.54      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.68      0.54      0.60      3219
          medical_help       0.58      0.20      0.30       638
      medical_products       0.71      0.29      0.41       418
     search_and_rescue       0.60      0.24      0.34       192
              security       0.20      0.01      0.03       144
              military       0.48      0.24      0.32       245
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.57      0.67       500
                  food       0.80      0.79      0.80       878
               shelter       0.79      0.54      0.64       705
              clothing       0.70      0.42      0.52       115
                 money       0.55      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [ ]:
import pickle
filename='final_model.pkl'
with open(filename,'wb') as file:
    pickle.dump(cv_new)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.